# Muye Day 1: Generating Unique id in IFLS 5

In [ ]:
library(haven)
library(dplyr)
bk_ar1 <- read_dta("C:/Users/Muflih/Downloads/hh14_all_dta/bk_ar1.dta")
View(bk_ar1)

#Check the length of hhid14 & pid14 digits
check_hhid14 <- bk_ar1 |>
  mutate(check = nchar(hhid14)) #To check the length of the variable character
mean(check_hhid14$check)
max(check_hhid14$check)

check_pid14 <- bk_ar1 |>
  mutate(check = nchar(pid14)) #To check the length of the variable character
mean(check_pid14$check)
max(check_pid14$check)

#Create new variable
bk_ar1 <- bk_ar1 |>
  mutate(id_new = paste(hhid14, sprintf("%02d", pid14), sep = ""))

#Validation check, if all id is unique or otherwise
validation_check <- bk_ar1 |>
  summarize(
    distinct_count = n_distinct(id_new),  # Count of unique `id_new`
    total_count = n()                     # Total count of rows
  ) |>
  mutate(difference = total_count - distinct_count)  # Calculate the difference

ifelse(validation_check$difference == 0, "all unique", "not unique")

print(validation_check)

# Muye Day 1: Mutating Common Useful Variables

In [ ]:
# b3a_d11 making years of schooling ---------------------------------------

b3a_dl1 <- read_dta("C:/Users/muham/Documents/sok produktif/Stata/work stations/IFLS/IFLS5/hh14_all_dta/b3a_dl1.dta")
View(b3a_dl1)
#Create new variable
b3a_dl1 <- b3a_dl1 |>
  mutate(id_new = paste(hhid14, sprintf("%02d", pid14), sep = ""))

##Recode d102 variable
b3a_dl1 <- b3a_dl1 |>
  mutate(read = case_when(
    dl02 == 3 ~ 0,
    dl02 == 8 ~ NA_real_, #8 become NA
    is.na(dl02) ~ NA_real_, #NA become NA
    TRUE ~ 1 #Everything else become 1
))

#Label the 'read' variable
attr(b3a_dl1$read, "label") <- "Able to read newspaper in Indonesian"
attr(b3a_dl1$read, "labels") <- c("No" = 0, "Yes" = 1)

#Check data validity
b3a_dl1 |>
  select(read) |>
  distinct() #Checking all categories in the variable

b3a_dl1 |>
  group_by(read) |> #Groping b3a_dl1 samples given the read variable
  summarize(n=n()) #Counting n per "read" category


##Recode d103 variable
b3a_dl1 <- b3a_dl1 |>
  mutate(write = case_when(
    dl03 == 3 ~ 0,
    dl03 == 8 ~ NA_real_, #8 become NA
    is.na(dl03) ~ NA_real_, #NA become NA
    TRUE ~ 1
  ))

#Label the 'write' variable
attr(b3a_dl1$write, "label") <- "Able to write letter in Indonesian"
attr(b3a_dl1$write, "labels") <- c("No" = 0, "Yes" = 1)

#Check data validity
b3a_dl1 |>
  select(write) |>
  distinct()

b3a_dl1 |>
  group_by(write) |>
  summarize(n=n())


##Recode d104 and dl07a variables -> If the sample is ever attending school
b3a_dl1$a777 <- 0 #Creating the middle variable where sample is currently attending shcool

for (x in 1:length(b3a_dl1$pid14)) {
  if (b3a_dl1$dl07a[x] == 1 & is.na(b3a_dl1$dl07a[x])==FALSE) {
    b3a_dl1$a777[x] <- 1
  } else if (is.na(b3a_dl1$dl07a[x])){
    b3a_dl1$a777[x] <- 0
  } else {
    b3a_dl1$a777[x] <- 0
  }
} #For all observation in b3a_dl1$a777: If dl07a=1 & is not missing, a777=1; If dl07a is missing, a777=0; If dl07a is otherwise, a777=0

b3a_dl1$everschl <- if_else(b3a_dl1$a777 == 1, b3a_dl1$everschl <- 1, b3a_dl1$everschl <- case_when(
  b3a_dl1$dl04 == 1 ~ 2,
  b3a_dl1$dl04 == 3 ~ 0,
  b3a_dl1$dl04 == 8 ~ NA_real_,
  is.na(b3a_dl1$dl04) ~ NA_real_,
)) #If a777=1 (currently attending school), everschl=1. If a777 is otherwise: everschl=2 if dl04=1 (ever attending school); everschl=0 if dl04=3 (never attending school); everschl=NA if dl04 is otherwise

#Label the 'everschl' variable
attr(b3a_dl1$everschl, "label") <- "Able to write letter in Indonesian"
attr(b3a_dl1$everschl, "labels") <- c("Never attended school" = 0,
                                      "Currently attending school" = 1,
                                      "Ever attended school" = 2)

#Check data validity
b3a_dl1 |>
  select(everschl) |>
  distinct()

b3a_dl1 |>
  group_by(everschl) |>
  summarize(n=n())


##Recode dl06 dl07 -> For level of eduction
b3a_dl1 <- b3a_dl1 %>%
  mutate(educlvl = case_when(
    # condition 1: no schooling
    (everschl == 0 | dl06 == 90 | (dl06 %in% c(2, 11, 72) & (dl07 < 7 | dl07 >= 98))) ~ 0,

    # condition 2: primary school
    ((dl06 %in% c(2, 11, 72) & dl07 == 7) | (dl06 %in% c(3, 4, 12, 73) & (dl07 < 7 | dl07 >=98)))~1,

    # condition 3: junior high school
    ((dl06 %in% c(3, 4, 12, 73) & dl07 == 7) | (dl06 %in% c(5, 6, 15, 74) & (dl07 < 7 | dl07 >= 98))) ~ 2,

    # condition 4: senior high school
    ((dl06 %in% c(5, 6, 15, 74) & dl07 == 7) | (dl06 %in% c(60, 61, 13) & (dl07 < 7 | dl07 >= 98))) ~ 3,

    # condition 5: higher education
    ((dl06 %in% c(60, 61, 62, 63, 13) & dl07 == 7) | (dl06 %in% c(62, 63) & (dl07 < 7 | dl07 >= 98))) ~ 4,

    # condition 6: Other categories
    dl06 == 95 ~ 10,
    dl06 == 14 ~ 14,
    dl06 == 17 ~ 17,

    # Missing values
    dl06 == 98 ~ NA_real_,  # .d in Stata (don't know)
    dl06 == 99 ~ NA_real_,  # .m in Stata (missing by design)
    TRUE ~ NA_real_  # Default for other cases (missing)
  ))

# Label the educlvl variable
attr(b3a_dl1$educlvl, "label") <- "Highest education level finished"

# Define value labels for educlvl
educlvl_labels <- c(
  "0" = "No schooling/not finished primary school",
  "1" = "Primary school/equivalent",
  "2" = "Junior high school/equivalent",
  "3" = "Senior high school/equivalent",
  "4" = "Higher education",
  "7" = "Madrasa",
  "10" = "Other",
  "14" = "Pesantren",
  "17" = "School for disabled"
)

# Apply value labels to educlvl
b3a_dl1 <- b3a_dl1 |>
  mutate(educlvl = factor(educlvl, levels = names(educlvl_labels), labels = educlvl_labels))

# Check the result
b3a_dl1 |>
  group_by(educlvl) |>
  summarize(n=n())

typeof(b3a_dl1$educlvl)


##Years of educ
#educ
b3a_dl1 <- b3a_dl1 %>%
  mutate(educ = case_when(
    # year 0–5
    (everschl == 0 | dl06 == 90 | (dl06 %in% c(2, 11, 72) & (dl07 %in% c(0, 98, 99)))) ~ 0,
    (dl06 %in% c(2, 11, 72) & dl07 %in% c(1:5)) ~ dl07,

    # year 6—8
    (((dl06 %in% c(2, 11, 72)) & (dl07 %in% c(6:7))) | ((dl06 %in% c(3, 4, 12, 73)) & (dl07 %in% c(0, 98, 99)))) ~ 6,
    ((dl06 %in% c(3, 4, 12, 73) & dl07 %in% c(1:2))) ~ 6 + dl07,
    # year 9—11
    (((dl06 %in% c(3, 4, 12, 73)) & (dl07 %in% c(3:7))) | ((dl06 %in% c(5, 6, 15, 74)) & (dl07 %in% c(0, 98, 99)))) ~ 9,
    ((dl06 %in% c(5, 6, 15, 74)) & (dl07 %in% c(1, 2))) ~ 9 + dl07,
    # year 12
    ((dl06 %in% c(5, 6, 15, 74)) & (dl07 %in% c(3:7))) ~ 12,
    ((dl06 %in% c(60, 61, 13)) & (dl07 %in% c(0, 98, 99))) ~ 12,

    # year 13
    ((dl06 %in% c(60)) & (dl07 %in% c(1))) ~ 13,

    # year 14
    ((dl06 %in% c(60)) & (dl07 %in% c(2:7))) ~ 14,

    # year 13—15
    ((dl06 %in% c(61, 13)) & (dl07 %in% c(1:3))) ~ 12 + dl07,

    # year 16
    (((dl06 %in% c(61, 13)) & (dl07 %in% c(4:7))) | ((dl06 %in% c(62)) & (dl07 %in% c(0, 98, 99)))) ~ 16,

    # year 17
    ((dl06 %in% c(62)) & (dl07 %in% c(1))) ~ 17,

    # year 18
    (((dl06 %in% c(62)) & (dl07 %in% c(2:7))) | ((dl06 %in% c(63)) & (dl07 %in% c(0, 98, 99)))) ~ 18,

    # year 18 + dl07
    ((dl06 %in% c(63)) & (dl07 %in% c(1:3))) ~ 18 + dl07,

    # year 22
    ((dl06 %in% c(63)) & (dl07 %in% c(4:7))) ~ 22,

    # Missing values
    dl06 == 98 ~ NA_real_,  # .d in Stata (don't know)
    (educlvl %in% c(7, 10, 14, 17)) | (dl06 == 99) ~ NA_real_,  # .m in Stata (missing by design)
    TRUE ~ NA_real_  # Default for other cases (missing)
  ))

# Label the educ variable
attr(b3a_dl1$educ, "label") <- "Years of education"


# Muye Day 1: Merging Book

In [ ]:
## remove irrelevant variable
#first dataset
bk_ar1 <- bk_ar1 |>
  select(id_new, ar02b, ar13, pidlink) #Selecting variables chosen from the bk_ar1 dataset

#second dataset
b3a_dl1 <- b3a_dl1 |>
  select(id_new, read, write, everschl, educlvl) #Selecting variables chosen from the b3a_dl1 dataset

merge <- merge(bk_ar1, b3a_dl1,by="id_new", all=TRUE) #Merging datasets based on "id_new" variable

length(merge$id_new) #Check the length of the new merged dataset

##Incase the base variable has different name
#Rename to match other dataset base variable
bk_ar1_1 <- bk_ar1 %>%
  rename(id = id_new)

#Just merge directly
merge2 <- merge(bk_ar1_1, b3a_dl1, by.x="id", by.y="id_new", all=TRUE) #X is the name of base variable on bk_ar1_1, and y is from b3a_dl1. all=TRUE means all not matched variables is still included

# Muye Day 1: Reshape from Long to Wide

In [ ]:
# reshape from long to wide. And also rowtotal to make total food consumption

b1_ks1 <- read_dta("R/Data for Learning/b1_ks1.dta")

consump <- b1_ks1 |>
  pivot_wider(
    id_cols = hhid14, #Variable that intended to be changed from long to wide
    names_from = ks1type, #New column names source
    values_from = ks02 #Source of value in the new columns
  )

View(consump)

#Create variable that summing all of the new widen variables
consump <- consump |>
  mutate(
    food_consump = rowSums(consump[2:length(consump)], na.rm = TRUE) #Food consumption is the total from the second row to the end
  )

#Check the descriptive statistics
mean(consump$food_consump)
max(consump$food_consump)
min(consump$food_consump)
var(consump$food_consump)
sd(consump$food_consump)


# Muye Day 2: Merging Waves

In [ ]:
##Prepare the first wave (IFLS 4) -----
library(haven)
library(dplyr)
bk_ar1_07 <- read_dta("C:/Muflih Irfan/Academic/College/R/hh07_all_dta/bk_ar1.dta")
View(bk_ar1_07)

#Firstly, create unique id for the bk_ar1_07
bk_ar1_07 <- bk_ar1_07 |>
  mutate(id_new = paste(hhid07, sprintf("%02d", pid07), sep = ""))

#Secondly, create unique id for the b3a_dl1_07
b3a_dl1_07 <- read_dta("C:/Muflih Irfan/Academic/College/R/hh07_all_dta/b3a_dl1.dta")
View(b3a_dl1_07)
b3a_dl1_07 <- b3a_dl1_07 |>
  mutate(id_new = paste(hhid07, sprintf("%02d", pid07), sep = ""))

#Recode d102 variable
b3a_dl1_07 <- b3a_dl1_07 |>
  mutate(read = case_when(
    dl02 == 3 ~ 0,
    dl02 == 8 ~ NA_real_, #8 become NA
    is.na(dl02) ~ NA_real_, #NA become NA
    TRUE ~ 1 #Everything else become 1
))

#Label the 'read' variable
attr(b3a_dl1_07$read, "label") <- "Able to read newspaper in Indonesian"
attr(b3a_dl1_07$read, "labels") <- c("No" = 0, "Yes" = 1)

#Recode d103 variable
b3a_dl1_07 <- b3a_dl1_07 |>
  mutate(write = case_when(
    dl03 == 3 ~ 0,
    dl03 == 8 ~ NA_real_, #8 become NA
    is.na(dl03) ~ NA_real_, #NA become NA
    TRUE ~ 1
  ))

#Label the 'write' variable
attr(b3a_dl1_07$write, "label") <- "Able to write letter in Indonesian"
attr(b3a_dl1_07$write, "labels") <- c("No" = 0, "Yes" = 1)

#Recode d104 and dl07a variables -> If the sample is ever attending school
b3a_dl1_07$a777 <- 0 #Creating the middle variable where sample is currently attending shcool

for (x in 1:length(b3a_dl1_07$pid07)) {
  if (b3a_dl1_07$dl07a[x] == 1 & is.na(b3a_dl1_07$dl07a[x])==FALSE) {
    b3a_dl1_07$a777[x] <- 1
  } else if (is.na(b3a_dl1_07$dl07a[x])){
    b3a_dl1_07$a777[x] <- 0
  } else {
    b3a_dl1_07$a777[x] <- 0
  }} #For all observation in b3a_dl1_07$a777: If dl07a=1 & is not missing, a777=1; If dl07a is missing, a777=0; If dl07a is otherwise, a777=0

b3a_dl1_07$everschl <- if_else(b3a_dl1_07$a777 == 1, b3a_dl1_07$everschl <- 1, b3a_dl1_07$everschl <- case_when(
  b3a_dl1_07$dl04 == 1 ~ 2,
  b3a_dl1_07$dl04 == 3 ~ 0,
  b3a_dl1_07$dl04 == 8 ~ NA_real_,
  is.na(b3a_dl1_07$dl04) ~ NA_real_,
)) #If a777=1 (currently attending school), everschl=1. If a777 is otherwise: everschl=2 if dl04=1 (ever attending school); everschl=0 if dl04=3 (never attending school); everschl=NA if dl04 is otherwise

#Label the 'everschl' variable
attr(b3a_dl1_07$everschl, "label") <- "Able to write letter in Indonesian"
attr(b3a_dl1_07$everschl, "labels") <- c("Never attended school" = 0,
                                      "Currently attending school" = 1,
                                      "Ever attended school" = 2)

#Recode dl06 dl07 -> For level of eduction
b3a_dl1_07 <- b3a_dl1_07 |>
  mutate(educlvl = case_when(
    # condition 1: no schooling
    (everschl == 0 | dl06 == 90 | (dl06 %in% c(2, 11, 72) & (dl07 < 7 | dl07 >= 98))) ~ 0,

    # condition 2: primary school
    ((dl06 %in% c(2, 11, 72) & dl07 == 7) | (dl06 %in% c(3, 4, 12, 73) & (dl07 < 7 | dl07 >=98)))~1,

    # condition 3: junior high school
    ((dl06 %in% c(3, 4, 12, 73) & dl07 == 7) | (dl06 %in% c(5, 6, 15, 74) & (dl07 < 7 | dl07 >= 98))) ~ 2,

    # condition 4: senior high school
    ((dl06 %in% c(5, 6, 15, 74) & dl07 == 7) | (dl06 %in% c(60, 61, 13) & (dl07 < 7 | dl07 >= 98))) ~ 3,

    # condition 5: higher education
    ((dl06 %in% c(60, 61, 62, 63, 13) & dl07 == 7) | (dl06 %in% c(62, 63) & (dl07 < 7 | dl07 >= 98))) ~ 4,

    # condition 6: Other categories
    dl06 == 95 ~ 10,
    dl06 == 14 ~ 14,
    dl06 == 17 ~ 17,

    # Missing values
    dl06 == 98 ~ NA_real_,  # .d in Stata (don't know)
    dl06 == 99 ~ NA_real_,  # .m in Stata (missing by design)
    TRUE ~ NA_real_  # Default for other cases (missing)
  ))

# Label the educlvl variable
attr(b3a_dl1_07$educlvl, "label") <- "Highest education level finished"

# Define value labels for educlvl
educlvl_labels <- c(
  "0" = "No schooling/not finished primary school",
  "1" = "Primary school/equivalent",
  "2" = "Junior high school/equivalent",
  "3" = "Senior high school/equivalent",
  "4" = "Higher education",
  "7" = "Madrasa",
  "10" = "Other",
  "14" = "Pesantren",
  "17" = "School for disabled"
)

# Apply value labels to educlvl
b3a_dl1_07 <- b3a_dl1_07 |>
  mutate(educlvl = factor(educlvl, levels = names(educlvl_labels), labels = educlvl_labels))

##Years of educ
#educ
b3a_dl1_07 <- b3a_dl1_07 |>
  mutate(educ = case_when(
    # year 0–5
    (everschl == 0 | dl06 == 90 | (dl06 %in% c(2, 11, 72) & (dl07 %in% c(0, 98, 99)))) ~ 0,
    (dl06 %in% c(2, 11, 72) & dl07 %in% c(1:5)) ~ dl07,
    # year 6—8
    (((dl06 %in% c(2, 11, 72)) & (dl07 %in% c(6:7))) | ((dl06 %in% c(3, 4, 12, 73)) & (dl07 %in% c(0, 98, 99)))) ~ 6,
    ((dl06 %in% c(3, 4, 12, 73) & dl07 %in% c(1:2))) ~ 6 + dl07,
    # year 9—11
    (((dl06 %in% c(3, 4, 12, 73)) & (dl07 %in% c(3:7))) | ((dl06 %in% c(5, 6, 15, 74)) & (dl07 %in% c(0, 98, 99)))) ~ 9,
    ((dl06 %in% c(5, 6, 15, 74)) & (dl07 %in% c(1, 2))) ~ 9 + dl07,
    # year 12
    ((dl06 %in% c(5, 6, 15, 74)) & (dl07 %in% c(3:7))) ~ 12,
    ((dl06 %in% c(60, 61, 13)) & (dl07 %in% c(0, 98, 99))) ~ 12,
    # year 13
    ((dl06 %in% c(60)) & (dl07 %in% c(1))) ~ 13,
    # year 14
    ((dl06 %in% c(60)) & (dl07 %in% c(2:7))) ~ 14,
    # year 13—15
    ((dl06 %in% c(61, 13)) & (dl07 %in% c(1:3))) ~ 12 + dl07,
    # year 16
    (((dl06 %in% c(61, 13)) & (dl07 %in% c(4:7))) | ((dl06 %in% c(62)) & (dl07 %in% c(0, 98, 99)))) ~ 16,
    # year 17
    ((dl06 %in% c(62)) & (dl07 %in% c(1))) ~ 17,
    # year 18
    (((dl06 %in% c(62)) & (dl07 %in% c(2:7))) | ((dl06 %in% c(63)) & (dl07 %in% c(0, 98, 99)))) ~ 18,
    # year 18 + dl07
    ((dl06 %in% c(63)) & (dl07 %in% c(1:3))) ~ 18 + dl07,
    # year 22
    ((dl06 %in% c(63)) & (dl07 %in% c(4:7))) ~ 22,
    # Missing values
    dl06 == 98 ~ NA_real_,  # .d in Stata (don't know)
    (educlvl %in% c(7, 10, 14, 17)) | (dl06 == 99) ~ NA_real_,  # .m in Stata (missing by design)
    TRUE ~ NA_real_  # Default for other cases (missing)
  ))

#Label the educ variable
attr(b3a_dl1_07$educ, "label") <- "Years of education"

##Chose only relevant variables
#First dataset
bk_ar1_07 <- bk_ar1_07 |>
  select(id_new, ar02b, ar13, pidlink) #Selecting variables chosen from the bk_ar1 dataset

#second dataset
b3a_dl1_07 <- b3a_dl1_07 |>
  select(id_new, read, write, everschl, educlvl) #Selecting variables chosen from the b3a_dl1 dataset

merge_07 <- merge(bk_ar1_07, b3a_dl1_07, by="id_new", all=TRUE) #Merging datasets based on "id_new" variable

##For the IFLS5, use previous steps in day 1 Muye

##Append the Wave 4 and 5
append <- rbind(merge, merge_07)




Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: Error: 'C:/Muflih Irfan/Academic/College/R/hh07_all_dta/bk_ar1.dta' does not exist.


# Muye Day 2: ARIMA Orang

In [ ]:
##call packages
library(quantmod)
library(tidyquant)
library(ggplot2)
library(dplyr)
library(forecast)
library(tseries)
library(TTR)
library(highcharter)

#BBRI
price_bbri <- getSymbols("BBRI.JK", auto.assign=FALSE, from="2014-01-01", to="2019-12-31")
head(price_bbri)

# Candle stick chart
chartSeries(price_bbri, name = "BBRI Price 2014-2019")

# Take only the closing price
closing_pr <- Cl(to.monthly(price_bbri))

# Decompose it
dc <- decompose(as.ts(closing_pr, start=c(2014,1)))
plot(dc)

highchart(type="stock") %>%
  hc_add_series(price_bbri) %>%
  hc_add_series(SMA(na.omit(Cl(price_bbri)),n=50),name="SMA(50)") %>%
  hc_add_series(SMA(na.omit(Cl(price_bbri)),n=200),name="SMA(200)") %>%
  hc_title(text="<b>BBRI Price Candle Stick Chart 2014-2019</b>")


# Fetch BBNI, BMRI, and IHSG stock prices
price_bbni <- getSymbols("BBNI.JK",auto.assign=FALSE,from="2014-01-01",to="2019-12-31")
price_bmri <- getSymbols("BMRI.JK",auto.assign=FALSE,from="2014-01-01",to="2019-12-31")
price_ihsg <- getSymbols("^JKSE",auto.assign=FALSE,from="2014-01-01",to="2019-12-31")

# Compare the stock prices
highchart(type="stock") %>%
  hc_add_series(Cl(price_bbri), name="BBRI") %>%
  hc_add_series(Cl(price_bbni), name="BBNI") %>%
  hc_add_series(Cl(price_bmri), name="BMRI") %>%
  hc_add_series(Cl(price_ihsg), name="IHSG") %>%
  hc_title(text="<b>BBRI vs BBNI vs BMRI vs IHSG Closing Price</b>")

# Calculate the stocks return
return_bbri <- dailyReturn(Cl(price_bbri))
return_bbni <- dailyReturn(Cl(price_bbni))
return_bmri <- dailyReturn(Cl(price_bmri))

# Combine the returns as one data frame
returns <- data.frame(return_bbri,return_bbni,return_bmri)
names(returns) <- c("return_bbri","return_bbni","return_bmri")
returns <- as.xts(returns)

# Plot the returns
library(PerformanceAnalytics)
charts.PerformanceSummary(returns,main="Daily Return BBRI vs BBNI vs BMRI 2014-2019")


# Forecast ----------------------------------------------------------------
# Number of period we want to forecast
n <- 100

# Splitting the data
train <- head(Cl(price_bbri), length(Cl(price_bbri))-n)
test <- tail(Cl(price_bbri), n)

##naive method

# Forecast the data
fc_na <- naive(train, h=n)

# Plot the result
autoplot(fc_na) +
  autolayer(ts(test, start=length(train)), series = "Test Data")

##ARIMA
# Create the Model
model_non <- auto.arima(train, seasonal=FALSE)

# Forecast n periods of the data
fc_non <- forecast(model_non, h=n)

# Plot the result
autoplot(fc_non)+
  autolayer(ts(test, start= length(train)), series="Test Data")

##SARIMA
# Create the Model
model_s <- auto.arima(train)

# Forecast n periods of the data
fc_s <- forecast(model_s, h=n)

# Plot the result
autoplot(fc_s)+
  autolayer(ts(test, start= length(train)), series="Test Data")

##evaluation
checkresiduals(fc_na)
checkresiduals(fc_non)

accuracy(fc_na)
accuracy(fc_non)


# Muye Day 2: ARIMA BBCA


In [ ]:
##call packages
library(seastests)
library(quantmod)
library(tidyquant)
library(ggplot2)
library(dplyr)
library(lubridate)
library(forecast)
library(scales)
library(Metrics)
library(tseries)

#BBCA
BBCA = tq_get("BBCA.JK",get = "stock.prices")

check_seasonality <- isSeasonal(BBCA$close, test = "combined", freq = 251)

if (check_seasonality == TRUE) {
  #Decomposing if there is seasonality
  data.ts <- ts(BBCA$close, frequency = 251)
  price.de <- decompose(data.ts)
  plot(price.de)

  BBCA$price <- price.de$random
  BBCA$close_price <- price.de$random

} else {
  BBCA$price <- BBCA$close
  BBCA$close_price <- BBCA$close
}

BBCA <- BBCA[!is.na(BBCA$price), ]

if (adf.test(BBCA$price)$p.value <= 0.01 & pp.test(BBCA$price)$p.value <= 0.01) {
  BBCA$price <- BBCA$price
  print("stationer at level")
  d = 0

} else {
  BBCA$price <- BBCA$price - lag(BBCA$price)
  BBCA <- BBCA[!is.na(BBCA$price), ]

  if (adf.test(BBCA$price)$p.value <= 0.01 & pp.test(BBCA$price)$p.value <= 0.01) {
    BBCA$price <- BBCA$price
    print("stationer at 1-diff")
    d = 1

  } else {
    BBCA$price <- BBCA$price - lag(BBCA$price)
    BBCA <- BBCA[!is.na(BBCA$price), ]

    if (adf.test(BBCA$price)$p.value <= 0.01 & pp.test(BBCA$price)$p.value <= 0.01) {
      BBCA$price <- BBCA$price
      print("stationer at 2-diff")
      d = 2

    } else {
      BBCA$price <- BBCA$price - lag(BBCA$price)
      BBCA <- BBCA[!is.na(BBCA$price), ]

      if (adf.test(BBCA$price)$p.value <= 0.01 & pp.test(BBCA$price)$p.value <= 0.01) {
        BBCA$price <- BBCA$price
        print("stationer at 3-diff")
        d = 3

      }
    }
  }
}

model <- tibble(p = 999999, d = 999999, q = 999999, aic = 999999, bic = 999999 )

for (p in 1:5) {
  for (q in 1:5) {
    arima <- Arima(BBCA$close, order=c(p,d,q))
    model <- model %>%
      add_row(
        p = p, d = d, q = q, aic = arima$aic, bic = arima$bic
      )
  }
}

model <- model[model$p != 999999, ]

model

View(model)

model <- model %>%
  mutate(
    both_aic_bic = (aic + bic)/2
  )

model <- model %>%
  mutate(
    min_aic = min(aic),
    min_bic = min(bic),
    min_both = min(both_aic_bic)
  )

rmse_calc <- tibble(
  actual_data_toforecast = BBCA$close[(length(BBCA$close)-9):length(BBCA$close)]
)

model <- model %>%
  mutate(
    rmse = 99999
  )

traing_set <- tibble(
  actual_data_training = BBCA$close[1:(length(BBCA$close)-9)]
)

rmse_calc <- tibble(
  actual_data_toforecast = BBCA$close[(length(BBCA$close)-9):length(BBCA$close)]
)

for (x in 1:length(model$p)) {
  if (model$aic[x] == model$min_aic[x]) {
    p1 = model$p[x]
    d1 = d
    q1 = model$q[x]
    best_model_1 <- Arima(traing_set$actual_data_training, order=c(p1,d1,q1))

    rmse_calc <- rmse_calc %>%
      mutate(model_1 = predict(best_model_1, 10)$pred)

    model <- model %>%
      add_row(
        p = p1, d = d1, q = q1, rmse = rmse(rmse_calc$actual_data_toforecast, rmse_calc$model_1)
      )
  }
}

for (x in 1:length(model$p)) {
  if (model$bic[x] == model$min_bic[x] & !is.na(model$bic[x])) {
    p2 = model$p[x]
    d2 = d
    q2 = model$q[x]
    best_model_2 <- Arima(traing_set$actual_data_training, order=c(p2,d2,q2))

    rmse_calc <- rmse_calc %>%
      mutate(model_2 = predict(best_model_2, 10)$pred)

    model <- model %>%
      add_row(
        p = p2, d = d2, q = q2, rmse = rmse(rmse_calc$actual_data_toforecast, rmse_calc$model_2)
      )
  }
}

for (x in 1:length(model$p)) {
  if (model$both_aic_bic[x] == model$min_both[x] & !is.na(model$min_both[x])) {
    p3 = model$p[x]
    d3 = d
    q3 = model$q[x]
    best_model_3 <- Arima(traing_set$actual_data_training, order=c(p3,d3,q3))

    rmse_calc <- rmse_calc %>%
      mutate(model_3 = predict(best_model_3, 10)$pred)

    model <- model %>%
      add_row(
        p = p3, d = d3, q = q3, rmse = rmse(rmse_calc$actual_data_toforecast, rmse_calc$model_3)
      )
  }
}

model <- model %>%
  mutate(rmse_cek = min(rmse))

for (x in 1:length(model$p)) {
  if (model$rmse_cek[x] == model$rmse[x]) {
    p_all = model$p[x]
    d_all = d
    q_all = model$q[x]
  }
}

best_model_all <- Arima(BBCA$close, order=c(p_all,d_all,q_all))

forecast_5t <- tibble(
  time_forward = c(1:5),
  forecast = c(predict(best_model_all, 5)$pred)
)

View(forecast_5t)

forecast_5t


ERROR: Error in library(tidyquant): there is no package called ‘tidyquant’


# Muye Day 3: Monte Carlo Analysis